In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hardikjainiitg","key":"41db23fb519649c5ffb2ad7e1cbf72ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
 97% 486M/503M [00:01<00:00, 362MB/s]
100% 503M/503M [00:01<00:00, 446MB/s]


In [ ]:
!unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [ ]:
import pandas as pd

train_path = "/content/cnn_dailymail/train.csv"
val_path   = "/content/cnn_dailymail/validation.csv"
test_path  = "/content/cnn_dailymail/test.csv"

use_cols = ["article", "highlights"]

train_df = pd.read_csv(
    train_path,
    usecols=use_cols,
    nrows=15_000
)

val_df = pd.read_csv(
    val_path,
    usecols=use_cols,
    nrows=1_000
)

test_df = pd.read_csv(
    test_path,
    usecols=use_cols,
    nrows=1_000
)

print(train_df.shape, val_df.shape, test_df.shape)

(15000, 2) (1000, 2) (1000, 2)


In [ ]:
import re

_whitespace_re = r"\s+"

def clean_df(df):
    # Rename columns (cheap)
    df = df.rename(
        columns={
            "article": "input_text",
            "highlights": "target_text"
        }
    )

    # Fill NaNs ONCE (no astype)
    df["input_text"] = df["input_text"].fillna("")
    df["target_text"] = df["target_text"].fillna("")

    # Vectorized string ops (C-backed, memory efficient)
    df["input_text"] = (
        df["input_text"]
        .str.strip()
        .str.replace(_whitespace_re, " ", regex=True)
        .str.replace("\u200b", "", regex=False)
    )

    df["target_text"] = (
        df["target_text"]
        .str.strip()
        .str.replace(_whitespace_re, " ", regex=True)
        .str.replace("\u200b", "", regex=False)
    )

    return df

In [ ]:
train_df = clean_df(train_df)
val_df   = clean_df(val_df)
test_df  = clean_df(test_df)

In [ ]:
import gc
gc.collect()

545

In [ ]:
from datasets import Dataset, DatasetDict
import gc

dataset = DatasetDict()

# ---- TRAIN ----
dataset["train"] = Dataset.from_pandas(
    train_df,
    preserve_index=False
)
del train_df
gc.collect()

# ---- VALIDATION ----
dataset["validation"] = Dataset.from_pandas(
    val_df,
    preserve_index=False
)
del val_df
gc.collect()

# ---- TEST ----
dataset["test"] = Dataset.from_pandas(
    test_df,
    preserve_index=False
)
del test_df
gc.collect()

dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 15000
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 1000
    })
})

In [ ]:
from transformers import AutoTokenizer

model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_input_length = 1024
max_target_length = 128

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
def preprocess(batch):
    model_inputs = tokenizer(
        batch["input_text"],
        truncation=True,
        max_length=max_input_length,
        padding=False,   # 🔑 critical
    )

    labels = tokenizer(
        text_target=batch["target_text"],
        truncation=True,
        max_length=max_target_length,
        padding=False,   # 🔑 critical
    )

    labels["input_ids"] = [
        [(t if t != tokenizer.pad_token_id else -100) for t in seq]
        for seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(
    preprocess,
    batched=True,
    batch_size=128,   # controls RAM peak otherwise default batch size si full
    remove_columns=["input_text", "target_text"],
)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("/content/tokenized_cnn_dm")

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
del tokenized_dataset, dataset
gc.collect()

351

In [ ]:
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/tokenized_cnn_dm")

In [ ]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator=DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding="max_length",
    max_length=1024
)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Training on:", next(model.parameters()).device)

Training on: cuda:0


In [ ]:
# ============================
# 2️⃣ Training Arguments
# ============================
training_args = TrainingArguments(
    output_dir="./checkpoints",

    # 🔹 AUTO batch size finder
    per_device_train_batch_size=4,      # 🔥 start small
    gradient_accumulation_steps=8,    # effective batch grows automatically

    # 🔹 Evaluation
    per_device_eval_batch_size=1,
    eval_strategy="steps",
    eval_steps=1000,

    # 🔹 Training
    num_train_epochs=3,
    learning_rate=2e-5,

    # 🔹 Performance
    fp16=True,
    torch_compile=False,

    # 🔹 Logging / saving
    logging_steps=50,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,

    report_to="none",
)

In [ ]:
from transformers import (
    # TrainingArguments,
    Trainer,
)
# ================================
# 9️⃣ Trainer
# ================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

"""
🔹 BATCHING INSIDE TRAINER
- Training: uses per_device_train_batch_size
- Eval: uses per_device_eval_batch_size
- Eval is forward-pass only (loss)
"""



/tmp/ipython-input-2783507188.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


'\n🔹 BATCHING INSIDE TRAINER\n- Training: uses per_device_train_batch_size\n- Eval: uses per_device_eval_batch_size\n- Eval is forward-pass only (loss)\n'

In [ ]:

# ================================
# 🔟 Train (with evaluation)
# ================================
trainer.train()

Step,Training Loss,Validation Loss
1000,1.926800,1.859571


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1407, training_loss=1.996927758185001, metrics={'train_runtime': 4657.4285, 'train_samples_per_second': 9.662, 'train_steps_per_second': 0.302, 'total_flos': 2.74381406208e+16, 'train_loss': 1.996927758185001, 'epoch': 3.0})

In [ ]:
save_dir = "./final_model"

# Save model
trainer.save_model(save_dir)

# Save tokenizer
tokenizer.save_pretrained(save_dir)

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [ ]:
!zip -r final_model.zip final_model

  adding: final_model/ (stored 0%)
  adding: final_model/special_tokens_map.json (deflated 52%)
  adding: final_model/tokenizer_config.json (deflated 75%)
  adding: final_model/training_args.bin (deflated 53%)
  adding: final_model/config.json (deflated 64%)
  adding: final_model/merges.txt (deflated 53%)
  adding: final_model/vocab.json (deflated 59%)
  adding: final_model/model.safetensors (deflated 7%)
  adding: final_model/tokenizer.json (deflated 82%)
  adding: final_model/generation_config.json (deflated 45%)


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3cab867fb5c12c66637da1123f6ba435ba92ee9988fefd305955fe9df124b60d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import torch
import evaluate
from tqdm import tqdm
from transformers import DataCollatorForSeq2Seq

# -----------------------------
# 1️⃣ Setup
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

rouge = evaluate.load("rouge")

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,           # dynamic padding
    return_tensors="pt",
)

# -----------------------------
# 2️⃣ Evaluation function
# -----------------------------
def evaluate_tokenized_dataset(
    dataset,
    model,
    tokenizer,
    batch_size=2,
    max_output_length=128,
    num_beams=2,
):
    predictions = []
    references = []

    column_names = dataset.column_names

    for start in tqdm(range(0, len(dataset), batch_size)):

        # Convert column-batch → list of samples
        features = [
            {k: dataset[k][i] for k in column_names}
            for i in range(start, min(start + batch_size, len(dataset)))
        ]

        # Pad batch
        batch = data_collator(features)

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]          # CPU tensor

        # 🔑 FIX: replace -100 before decoding
        labels = labels.clone()
        labels[labels == -100] = tokenizer.pad_token_id

        with torch.no_grad():
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                generated_ids = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=max_output_length,
                    num_beams=num_beams,
                    length_penalty=0.8,
                )

        preds = tokenizer.batch_decode(
            generated_ids,
            skip_special_tokens=True,
        )
        refs = tokenizer.batch_decode(
            labels,
            skip_special_tokens=True,
        )

        predictions.extend(preds)
        references.extend(refs)

    return rouge.compute(
        predictions=predictions,
        references=references,
    )

# -----------------------------
# 3️⃣ Run evaluation
# -----------------------------
scores = evaluate_tokenized_dataset(
    dataset=tokenized_dataset["test"],
    model=model,
    tokenizer=tokenizer,
    batch_size=2,          # safe for T4
    max_output_length=128,
    num_beams=2,
)

print(scores)


100%|██████████| 500/500 [08:55<00:00,  1.07s/it]


{'rouge1': np.float64(0.395668529454663), 'rouge2': np.float64(0.1803651702810095), 'rougeL': np.float64(0.2758795871348264), 'rougeLsum': np.float64(0.27564607797554386)}


In [ ]:
test_df = pd.read_csv(
    test_path,
    usecols=use_cols,
    nrows=1
)

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# -----------------------------
# 1️⃣ Load model & tokenizer
# -----------------------------
MODEL_DIR = "/content/final_model"   # your saved folder

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)

device = 0 if torch.cuda.is_available() else -1

# -----------------------------
# 2️⃣ Create pipeline
# -----------------------------
pipe = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device,   # GPU if available
)

# -----------------------------
# 3️⃣ Generation config
# -----------------------------
gen_kwargs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128,
}

# -----------------------------
# 4️⃣ Predict
# -----------------------------
# Convert Series to list of strings for the pipeline
sample_text_input = test_df["article"].tolist()
reference_output = test_df["highlights"].iloc[0]
sample_text_output = test_df["article"].iloc[0]

print("Dialogue:\n")
print(sample_text_output)

print("\nReference Summary:\n")
print(reference_output)

print("\nModel Prediction:\n")
output = pipe(sample_text_input, **gen_kwargs)
print(output[0]["summary_text"])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dialogue:

Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting

In [ ]:
test="Virat Kohli once again proved his importance to Indian cricket after playing a crucial role in India’s recent international match, where his calm batting and experience helped stabilize the team during a difficult phase. After a brief period of inconsistent form earlier in the season, Kohli showed signs of a strong comeback by building his innings patiently and guiding younger players at the crease. Experts praised his fitness, focus, and ability to handle pressure, noting that his presence alone boosts team confidence. Fans and former players also highlighted how Kohli’s leadership, even without the captaincy, continues to influence India’s performance across formats. With several major tournaments approaching, selectors and supporters believe Kohli’s form will be a key factor in India’s success."
output = pipe(test, **gen_kwargs)
output

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'summary_text': 'Virat Kohli once again proved his importance to Indian cricket after playing a crucial role in India’s recent international match . After a brief period of inconsistent form earlier in the season, Kohli showed signs of a strong comeback by building his innings patiently and guiding younger players at the crease .'}]